# [1] Define the model

In [92]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [93]:
query = "I just discovered the course. Can I still join it?"

## 1.1. First value of resulting vector after embendding

In [94]:
embedding_model.encode(query)[0]

0.07822261

In [95]:
v = embedding_model.encode(query)

## 1.2. Number of parameters for embedding

In [96]:
len(v)

768

# [2] Get the documents

In [97]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

## 2.1 Filter by course

In [98]:
documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

In [99]:
len(documents)

375

## 2.2 Documents vectorization (question/ text)

In [100]:
embeddings = []

for doc in documents:
    qa_text = f'{doc['question']} {doc['text']}'
    embeddings.append(embedding_model.encode(qa_text))

In [101]:
import numpy as np

X = np.array(embeddings)
X.shape
# N: Docs, N: Vectors per doc

(375, 768)

In [102]:
v.dot(v)

0.9999999

## 2.3 Max Score for q based on documents (cosine similarity)

In [103]:
scores = X.dot(v)

In [104]:
max(scores)

0.6506574

# [3] Simple Vector Search 

## 3.1 Define simple vector search

In [105]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=3) # Max Score = 0.65065 (1-st document)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## 3.2 Define Elastic Search

In [106]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = 'course-questions'

es_client.indices.delete(index = index_name, ignore_unavailable = True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [107]:
i = 0
for doc in documents:
    doc['question_text_vector'] = X[i, :]
    i = i + 1
    es_client.index(index=index_name, document=doc)

In [108]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [109]:
i = 0
for doc in documents:
    doc['question_text_vector'] = X[i, :]
    i = i + 1
    es_client.index(index=index_name, document=doc)

In [110]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
elastic_search_knn("question_text_vector", v, "machine-learning-zoomcamp")

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the

# [4] Hit-rate for our Simple Vector Search

## 4.1 Hit-rate for our Simple Vector Search

In [111]:
import pandas as pd

df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [112]:
ground_truth[:3]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [113]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

## 4.2 Calculate hit-rate for Simple Vector Search

In [114]:
relevance_total = []

for query in ground_truth:
    doc_id = query['document']
    results = search_engine.search(v_query = embedding_model.encode(query['question']), num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [115]:
hit_rate(relevance_total)

0.9398907103825137

## 4.3 Calculate hit-rate for Elastic Vector Search

In [116]:
relevance_total = []

for query in ground_truth:
    doc_id = query['document']
    results = elastic_search_knn("question_text_vector", embedding_model.encode(query['question']), "machine-learning-zoomcamp")
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [117]:
hit_rate(relevance_total)

0.9103825136612022